In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from scwidgets import (AnswerRegistry, TextareaAnswer, CodeDemo,
                       ParametersBox, PyplotOutput, ClearedOutput,
                       AnimationOutput,CodeCheckerRegistry, GLOBAL_TRAITS)
#no more  CodeChecker
from widget_code_input import WidgetCodeInput
from ipywidgets import Layout, Output, Textarea
from ase import Atoms

In [3]:
#### AVOID folding of output cell 

In [4]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

# Introduction to atomic-scale modeling

This course provides a hands-on introduction to the modeling of materials at the atomic scale. 
The purpose is both to provide a first example of the application of some of the techniques used to predict and characterize the behavior of materials in terms of the fundamental physical laws that govern the interactions between their atoms, as well as to demonstrate in a practical, numerical way some of the foundational concepts of materials science, such as the formation of crystal structures, 

The course is organized as a series of Jupyter notebooks, each covering one module:

1. [Atomic structures on a computer](./01-Atomic_scale_structures.ipynb)
2. [Crystallography and diffraction](./02-Crystallography.ipynb)
3. [Vibrations in solids, phonons](./03-Lattice_dynamics.ipynb)
4. [Interatomic potentials](./04-Potentials.ipynb)
5. [Defects in solids](./05-Defects.ipynb)
6. [Molecular dynamics](./06-Molecular_dynamics.ipynb)
7. [Machine learning](./07-Machine_learning.ipynb)

# Course how-to

The course contains a combination of text-book-style explanations, simple mathematical derivations, and interactive widgets in which you need to manipulate functions or atomic structures, and/or enter short snippets of code. The course material is conceived so that minimal amounts of prior knowledge about Python or Jupyter notebooks is needed, but you will have to do _some_ coding. Looking up examples and documentation online is fine, and even encouraged.

### Checking your work

All the courses modules have an accompanying `module_XX_checker.json` file containing tests for coding exercises. These tests are meant to help you control the correctness of your code and may be used for grading. 

Each coding exercises with checks have a "Check code" button, which you can click to control your answer before saving and/or submitting the notebook to grading.

__Do not move, erase of modify the__ `module_XX_checker.json` __file.__  If you have any problems, contact a teacher or teaching assistant.

In [5]:
GLOBAL_TRAITS.teacher_mode = True


In [6]:
check_registry = CodeCheckerRegistry('module_0_checker.json') 
display(check_registry)

CodeCheckerRegistry(children=(Output(layout=Layout(height='100%', width='100%')), Box(children=(Button(descrip…

### Saving your work
At the top of each notebook you will find a text box to enter your name. This will also be used to create a file in which you can save (and load from) the data you have entered into exercises and text widgets. We suggest to use `SurnameName`, which will lead to a file named `module_XX-SurnameName.json`. If you are looking at these notebooks as part of a formal course, you will be able to send these for grading. 

_NB: the state of the widgets is not saved automatically, so remember to click "Save" once you are done with a notebook_

In [7]:
answer_registry = AnswerRegistry(prefix="module_0")
display(answer_registry)

AnswerRegistry(children=(Output(layout=Layout(height='100%', width='100%')), HBox(children=(Dropdown(descripti…

In [8]:
module_summary = TextareaAnswer("general comments on this module", layout=Layout(width="100%"))
answer_registry.register_answer_widget("module-summary", module_summary)
display(module_summary)

TextareaAnswer(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--unsaved')), T…

<span style="color:blue">
<br><b>01</b> Throughout the notebook you will find highlighted paragraphs with a bold number.  These are instructions that you need to follow to answer some exercises, or comment on what you observe in interactive widgets. 
</span>

This might imply typing some open answer commenting the results you are seeing, or reporting the result of some simple derivation. Please keep this as tidy and clean: be concise and to the point, this is to help you collect your thoughts as much as to provide evaluation. 

# Appmode and dependencies

The notebooks use the *Appmode* Jupyter plugin, which you can activate by clicking on the corresponding button
<img src="figures/appmode_button.png" width="100"/>
and can be run with minimal knowledge of the Python language. Students with stronger programming background, who want to look "under the hood" can look directly at the source code before switching to Appmode to run them - or run the cells directly without activating the Appmode.

If you are using these notebooks as part of a class, hopefully you will be running them already in a fully-configured environment. If instead you are using them on your own system, you may need to install several prerequisite. From the main folder, run `pip install -r requirements.txt`, or a similar command with your favourite Python package manager

# Interactive widgets

The notebooks often contain interactive widgets that can be manipulated by changing some slider values, to visualize the concepts being discussed. Usually these don't require any coding, just to follow some instructions and/or to experiment with the values do develop a more intuitive understanding of the significance of an equation, or to test its limits.

In [9]:
def plot_sine(ax, w, a, abval):
    xgrid = np.linspace(0,5,100)
    ax = ax[0]
    if abval:
        ax.plot(xgrid, np.abs(a*np.sin(xgrid*w)))
        ax.set_ylabel(r"$|\sin x|$")
    else:
        ax.plot(xgrid, a*np.sin(xgrid*w))
        ax.set_ylabel(r"$\sin x$")
    ax.set_xlabel("$x$")
    

    

In [10]:
sine_parameterbox = ParametersBox(w = (2.0, 0, 10, 0.1, r'$\omega$'), a = (2., 1,100, 2, r'$A$'), 
                                      abval=(False, r'Absolute value'))
example_fig = plt.figure()
example_fig.add_subplot(111)
example_pyplot_output =  PyplotOutput(example_fig)
axes_example = example_pyplot_output.figure.get_axes()
plot_sine(axes_example,sine_parameterbox.value['w'],sine_parameterbox.value['a'],sine_parameterbox.value['abval'])

def update_visualizers(w,a, abval, visualizers):
    pyplot_output = visualizers[0]
    axes = pyplot_output.figure.get_axes()
    plot_sine(axes,w,a,abval)
p = CodeDemo(
            input_parameters_box = sine_parameterbox,
            visualizers=[example_pyplot_output],
            update_visualizers=update_visualizers
            ) 

A sine function $y = A \sin \omega x$ oscillates with a period $2\pi/\omega$ amd an amplitude spanning the range $[-A,A]$.

In [11]:
display(p)

CodeDemo(children=(HBox(children=(ParametersBox(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-b…

In [13]:
structure = Atoms("OH2", positions=[[0,0,0], [0.7,0.5,0],  [-0.7,0.5,0]])

cs1 = chemiscope.show([structure], mode="structure")

We can also display atomic structures in a dedicated [chemiscope](https://chemiscope.org/) widget.

In [14]:
display(cs1)

StructureWidget(value=None, data='{"meta": {"name": " "}, "structures": [{"size": 3, "names": ["O", "H", "H"],…

# Code widgets

Some exercises require inputting short code snippets into a dedicated code widget. This code forms the body of a function, whose return value can then be checked by plotting, or by comparison with known reference values. The function is compiled as a stand-alone python code, so you can only use variables and modules that are defined or imported within its scope.

In [15]:
GLOBAL_TRAITS.teacher_mode=True

cw = WidgetCodeInput(
        function_name="plot_function", 
        function_parameters="x, a, w, f_abs",
        code_theme = "default",
        docstring="""
Computes the sine function with a given frequency and amplitude, optionally taking the absolute value

:param x: the input variable
:param a: the amplitude
:param w: the frequency
:param f_abs: bool: take the absolute value?

:return: abs(a*sin(w x)) if f_abs else a*sin(w x)
""",
            function_body="""
# Write your solution, then click on the button below to update the plotter and check against the reference values
import numpy as np
return 0.0*np.array(x)
"""
        )

# Correct answer, only for testing purposes:
cw.function_body = """
import numpy as np
if not f_abs:
	return a*np.sin(w*x)
else:
	return np.abs(a*np.sin(w*x))
"""

#We prepare the visualization and their update callback:
fig2 = plt.figure()
fig2.add_subplot(111)
sine_pyplot_output_exercise =  PyplotOutput(fig2)

sine_parameterbox = ParametersBox(a = (2., 1,100, 2, r'$A$'), w = (2.0, 0, 10, 0.1, r'$\omega$'),
                                       abval=(False, r'Absolute value'))
axes_exercise = sine_pyplot_output_exercise.figure.get_axes()


def code_update_visualizer(a,w,abval,code_input,visualizers):
    pyplot_output = visualizers[0]
    axes = pyplot_output.figure.get_axes()
    ax = axes[0]
    sine_func = code_input.get_function_object()
    xgrid = np.linspace(0,5,100)
    ax.plot(xgrid,sine_func(xgrid,a,w,abval))
    ax.set_ylabel(r"$\sin x$")
    ax.set_xlabel("$x$")
    
#Initialize CodeDemo and initialize it's checks in the check_registry:
sine_code_demo = CodeDemo( 
            code_input = cw,
            code_checker=check_registry,
            input_parameters_box = sine_parameterbox,
            visualizers=[sine_pyplot_output_exercise],
            update_visualizers=code_update_visualizer,
            ) 

#This line is here only to clean checks for prototyping, not a part of the module:
check_registry.init_checks("sine_code_demo", sine_code_demo) # resets existing checks


#Option 1 : Add checks from with a json file. Teacher gives only input, 
#output is calculated (possibly hashed) and given to the check function via a json file
#To Alex: defining x as a list breaks the code because if x = [0] and w = 2, w*x = [0,0]. 
check_registry.check_from_file = True

input_parameters_exercise1 = [{"x" : 1, "a" : 1, "w" : 1, "abs" : True},
                              {"x" : np.pi/2, "a" : 2, "w" : 1, "abs" : True},
                              {"x" : 0, "a" : 3, "w" : 0.5, "abs" : False}]

                              
check_registry.add_check("sine_code_demo",
                         input_parameters_exercise1, equal_function=np.allclose) # np.allclose is automatically chosen
#Uncomment to check that checks are correctly created
#print(f"Checks succesfully created : {check_registry.create_check('sine_code_demo')}")


#Option 2 - Add checks by copy-pasting. Provide input and receive the output in a copy-pastable format
#Copy paste output to initialize the check function:
check_registry.check_from_file = False
check_registry.output_check_reference("sine_code_demo", input_parameters_exercise1)
check_registry.add_check_outputref("sine_code_demo", input_parameters_exercise1,
                                  pasted_output = [
np.array(0.8414709848078965),
np.array(2.0),
np.array(0.0),
] )

sine_code_demo.run_and_display_demo()

Copy paste this to check_registry.add_check_outputref():
[
np.array(0.8414709848078965),
np.array(2.0),
np.array(0.0),
]


CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box',)), CodeDemoBox(_dom_classes=('scwi…

These can also be combined with a structure visualizer to generate atomic structures

In [16]:
cw2 = WidgetCodeInput(
        function_name="write_structure", 
        function_parameters="",
        code_theme = "default",
        docstring="""
Generate an atomic structure for water, with a structure corresponding to the XYZ file
3
H2O molecule
O  0   0   0
H  0.7 0.5 0
H -0.7 0.5 0

:return: an ASE atoms object that can be visualized in a 3D structure viewer
""",
            function_body="""
# Write your solution, then click on the button below to update the plotter and check against the reference values
import ase

structure = ase.Atoms("OH2", positions=[[0,0,0],[0.7,0.5,0],[-0.7,0.5,0]])

# you can also print messages to debug (they will appear in the space below the widget)
#print("Structure symbols: ", structure.symbols)

# and even plot stuff if you feel like (uncomment to see this)
# import matplotlib.pyplot as plt
# plt.clf()
# plt.plot([0, 1, 2], [0, 1, 4], 'r*')
# plt.show()

# ... where errors will also appear (uncomment this line)
# p = ase.nonexistentcall

return structure
"""
        )

In [17]:
GLOBAL_TRAITS.teacher_mode=True

def cs_match(a, b):
    return (str(b.symbols) == a[0]) and np.allclose(b.positions, a[1])    



input_args = {
        (): ("OH2", [[0,0,0], [0.7,0.5,0],  [-0.7,0.5,0]]) }, 



def chemiscope_update_visualizers_structure(code_input, visualizers):
    cleared_output = visualizers[0]
    frame = code_input.get_function_object()()
    with cleared_output:
        chemiscope_widget = chemiscope.show(frames = [frame], mode="structure")
        display(chemiscope_widget)


    
def custom_equality_function(atom1, atom2):
    #Can have custom asserts:
    assert 8 == atom1.numbers[0], "The oxygen atom goes in the middle of the molecule!"
    assert 1 in atom1.numbers, "Hydrogen expected in the molecule. No hydrogen was found"
    #Must have a boolean return:
    return np.allclose(atom1.positions,atom2.positions) and np.allclose(atom1.numbers,atom2.numbers)

cs_code_demo = CodeDemo(
    code_input = cw2,
    code_checker= check_registry,
    visualizers=[ClearedOutput()],
    update_visualizers=chemiscope_update_visualizers_structure,
    update_on_input_parameter_change = False
)
check_registry.init_checks("cs_code_demo",cs_code_demo)
#Option 1 : Add checks from with a json file. Teacher gives only input, 
#output is calculated (possibly hashed) and given to the check function via a json file
#To Alex: defining x as a list breaks the code because if x = [0] and w = 2, w*x = [0,0]. 
check_registry.check_from_file = True
check_registry.add_check("cs_code_demo",{},equal_function=custom_equality_function)

#Option 2 - Add checks by copy-pasting. Provide input and receive the output in a copy-pastable format
#Copy paste output to initialize the check function:
check_registry.check_from_file = False
check_registry.output_check_reference("cs_code_demo", {})
check_registry.add_check_outputref("cs_code_demo", {},
                                  pasted_output = Atoms(symbols='OH2', pbc=False))

cs_code_demo.run_and_display_demo()

Copy paste this to check_registry.add_check_outputref():
Atoms(symbols='OH2', pbc=False)


CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box',)), CodeDemoBox(_dom_classes=('scwi…